In [1]:
from Parallelizer import make_parallel
from common import  Common
import re
import pandas as pd
import time
from urllib.parse import unquote
from utils import *
import json
from math import ceil


In [2]:
PATTERN=r"\b(?:[A-Za-z]+)?\d+"

# TIKTOK

In [3]:
import requests
from bs4 import BeautifulSoup
from time import sleep
count = 0

# @make_parallel
def request_tiktok(url):
    payload = {}
    headers = {
        'authority': 'oec-api.tiktokv.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '"Microsoft Edge";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        # 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.47'
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'
    }
    try_time = 0
    json_obj = None
    # url = SB_URL.format(url=url)
    while not json_obj:
        # url = f"https://oec-api.tiktokv.com/view/product/{product_id}"
        response = requests.request("GET", url, headers=headers, data=payload)
        try_time += 1
        print(f"{try_time}: {url}")
        try:
            soup = BeautifulSoup(response.text, 'html.parser')

            # get content of script id = RENDER_DATA
            script = soup.find('script', id='RENDER_DATA')
            # get content of script
            script_content = script.contents[0]
            # get data from script
            # print(response.text)
            json_obj = json.loads(unquote(script_content))
            # print(json_obj)
        except Exception as e:
            print(e)
            json_obj = None
            print(f"fail: {url} {response.text}")
            sleep(10)
    return json_obj
@make_parallel
def transform(response, results=[]):

    product_transformed = {
        "product_id": "failed",
        "name": "failed",
        "link": "failed",
        "type": "Sp lẻ",
        "min_price": "failed",
        "max_price": "failed",
        "sold_count": "failed",
        "min_original_price": "failed",
        "max_original_price": "failed",
    }

    for i in range(5):
        product_transformed[f"image_{i+1}"] = "failed"

    try:
        product_base = response['2']['initialData']['productInfo']['product_base']
        mapping_product_base = {
            "real_price": ["price", "real_price"],
            "original_price": ["price", "original_price"],
            "images": ["images"],
            "sold_count": ["sold_count"],
            "title": ["title"]
        }
        transformed_product_base = Common.mapping_data(
            product_base, mapping_product_base)
        product_id = response['2']['initialData']['productInfo']['product_id']
        print(product_id)
        name = transformed_product_base['title']
        price_pattern = re.compile(r'(\d+\.?\d*)')
        try:
            is_flash_sale = bool(
                response['2']['initialData']['productInfo'].get("flash_sale", False)
            )
            prices = price_pattern.findall(transformed_product_base['real_price'].replace('.', ''))
            if len(prices) == 2:
                min_price = int(prices[0].replace('.', ''))
                max_price = int(prices[1].replace('.', ''))
            else:
                min_price = None
                max_price = int(prices[0].replace('.', ''))
            original_price = price_pattern.findall(
                transformed_product_base['original_price'])
            if len(original_price) == 2:
                min_original_price = int(original_price[0].replace('.', ''))
                max_original_price = int(original_price[1].replace('.', ''))
            else:
                min_original_price = None
                if len(original_price) == 0:
                    max_original_price = None
                else:
                    max_original_price = int(original_price[0].replace('.', ''))
        except Exception as e:
            print("not found price")
            min_price = None
            max_price = None
            min_original_price = None
            max_original_price = None

        sold_count = transformed_product_base['sold_count']

        images = [None]*5
        for i,image in enumerate(transformed_product_base["images"][:5]):
            images[i] = f'=image("{image["thumb_url_list"][0]}")'

        product_transformed = {
            "product_id": product_id,
            "name": name.upper(),
            "link": f"https://oec-api.tiktokv.com/view/product/{product_id}",
            "type": "Sp lẻ",
            "min_price": min_price,
            "max_price": max_price,
            "sold_count": sold_count,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
        }
        
        if is_flash_sale:
            if min_original_price is not None:
                product_transformed["min_price"] = min_original_price
            if max_original_price is not None:
                product_transformed["max_price"] = max_original_price

        for i in range(5):
            product_transformed[f"image_{i+1}"] = images[i]

    except Exception as e:
        print(f"fail: {e}")
        if product_id:=response['_location'].split('/')[-1]:
            product_transformed["product_id"] = product_id
            product_transformed['link'] =  f"https://oec-api.tiktokv.com/view/product/{product_id}"
            print(f"failed {product_id}")

    results.append(product_transformed)

## CRALW

In [4]:
# product_infos = request_tiktok(links)
# len(product_infos), len(links)
df_shop = pd.read_csv('2512.csv')
for shop in df_shop['shop'].unique():
    links = df_shop[df_shop['shop'] == shop]['link'].tolist()
    product_infos = []
    for link in links:
        product_infos.append(request_tiktok(link))
        time.sleep(3)
        
    len(product_infos), len(links)
    results = []
    transform(product_infos, results)
    brand = shop

    df = pd.DataFrame(results)
    df['type'] = 'Sp lẻ'
    colums = ['product_id','name','link','type','min_price','max_price','sold_count','image_1','image_2','image_3','image_4','image_5','min_original_price','max_original_price','product_code']
    df.shape
    import re


    df['name'] = df['name'].apply(lambda x: str(x).replace('_', ' ').upper())
    df['name'] = df['name'].apply(lambda x: str(x).replace('-', '').upper())
    df['product_code'] = df['name'].apply(lambda x: re.findall(PATTERN, x)[-1] if len(re.findall(PATTERN, x)) > 0 else None)
    df['product_code'] = df['product_code'].apply(lambda x: x.upper() if x else None)
    # df['product_code'] = df['name'].apply(lambda x: x.split(' ')[-1])

    from pathlib import Path
    if not Path(f"data/{brand}").exists():
        Path(f"data/{brand}").mkdir(parents=True, exist_ok=True)
    
    df = df.sort_values(by=['name'])

        
    df.to_csv(f"data/{brand}/tiktok.csv", index=False, columns=colums)


1: https://oec-api.tiktokv.com/view/product/1729688813146245409
1: https://oec-api.tiktokv.com/view/product/1729680240148252961
1: https://oec-api.tiktokv.com/view/product/1729881371453393185
1: https://oec-api.tiktokv.com/view/product/1729436543111629089
1: https://oec-api.tiktokv.com/view/product/1729436543345330465
1: https://oec-api.tiktokv.com/view/product/1729436544224495905
1: https://oec-api.tiktokv.com/view/product/1729573352581990689
1: https://oec-api.tiktokv.com/view/product/1729730390153660705
1: https://oec-api.tiktokv.com/view/product/1729705588596312353
1: https://oec-api.tiktokv.com/view/product/1729749031445825825
1: https://oec-api.tiktokv.com/view/product/1729750987104684321
1: https://oec-api.tiktokv.com/view/product/1729541482493151521
1: https://oec-api.tiktokv.com/view/product/1729741872800631073
1: https://oec-api.tiktokv.com/view/product/1729610093319260449
1: https://oec-api.tiktokv.com/view/product/1729597976309041441
1: https://oec-api.tiktokv.com/view/prod

# SHOPE

In [ ]:
from pathlib import Path
all_items = []
with open(f"data/{brand}/all.json", "r", encoding="utf8") as f:
    all_cralwed = json.load(f)
    for crawled in all_cralwed:
        if data:=crawled.get('data'):
            all_items += crawled['data']['items']
        elif 'items' in crawled:
            all_items += crawled['items']
results = []


def transform_product(item):
    try:
        if "item_basic" in item:
            item = item['item_basic']
        product_id = item['itemid']
        shop_id = item['shopid']
        name = item['name']
        link = transform_link(name,shop_id,product_id)
        max_price = item['price_max'] /100000
        min_price = item['price_min'] / 100000 if item['price_min'] and item['price_min'] !=item['price_max'] else None
        sold_count = int(round(item['historical_sold']/100) * 100) if item['historical_sold'] >=1000 else item['historical_sold']
        max_original_price = item['price_max_before_discount'] / 100000 if item['price_max_before_discount'] > 0 else None
        min_original_price = item['price_min_before_discount'] / 100000 if item['price_min_before_discount'] >0 and item['price_min_before_discount'] != item['price_max_before_discount']  else None
        images = [None]*5
        for i,image in enumerate(item["images"][:5]):
            images[i] = f'=image("https://down-vn.img.susercontent.com/file/{image}")'
            
        product_transformed = {
            "product_id": product_id,
            "name": name,
            "link": link,
            "type": "Sp lẻ",
            "min_price": min_price,
            "max_price": max_price,
            "sold_count": sold_count,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
        }
        
        for i in range(5):
            product_transformed[f"image_{i+1}"] = images[i]
        return product_transformed
    except Exception as e:
        print(e)
        print(json.dumps(item, indent=4))
    

In [ ]:
transformed_products = []
for item in all_items:
    transformed_products.append(transform_product(item))

In [ ]:
df = pd.DataFrame(transformed_products)
columns = ["product_id","name","link","type","min_price","max_price","sold_count","image_1","image_2","image_3","image_4","image_5","min_original_price","max_original_price","product_code"]

df['type'] = 'Sp lẻ'
df['name'] = df['name'].apply(lambda x: x.replace('_', ' ').upper())
df['name'] = df['name'].apply(lambda x: x.replace('-', '').upper())
df['product_code'] = df['name'].apply(lambda x: re.findall(PATTERN, x)[-1] if len(re.findall(PATTERN, x)) > 0 else None)
df['product_code'] = df['product_code'].apply(lambda x: x.upper() if x else None)
print(df.shape)
df.to_csv(f"data/{brand}/shopee.csv", index=False, columns=columns)

In [ ]:
df_tt = pd.read_csv(f"data/{brand}/tiktok.csv")
df_shopee = pd.read_csv(f"data/{brand}/shopee.csv")
def text_comparision(text1, text2):
    text1 = text1.lower()
    text2 = text2.lower()
    text1 = text1.replace('_', ' ')
    text2 = text2.replace('_', ' ')
    set1 = set(text1.split(' '))
    set2 = set(text2.split(' '))
    # return len(set1.intersection(set2)) / len(set1.union(set2))
    return len(set1.intersection(set2)) 

def find_most_similar_name(name,price, df_shopee):
    df_shopee['similarity'] = df_shopee.apply(lambda x: text_comparision(name, x['name']), axis=1)
    df_shopee = df_shopee.sort_values(by=['similarity'], ascending=False)
    max_similarity = df_shopee.iloc[0]['similarity']
    max_index_top_matches = 0
    for i in range(len(df_shopee)):
        if df_shopee.iloc[i]['similarity'] < max_similarity:
            max_index_top_matches = i
            break
        
    # print(max_index_top_matches)
    # print(df_shopee[["similarity", "name"]])
    if max_index_top_matches <= 0:
        return df_shopee.iloc[0]['name']
    
    # if type(price) == str:
    #     print(price)
    #     print(name)
    #     return df_shopee.iloc[0]['name']
        
    df_shopee = df_shopee.iloc[:max_index_top_matches]
    df_shopee['price_diff'] = df_shopee.apply(lambda x: abs(float(price) - x['max_price']), axis=1)
    df_shopee = df_shopee.sort_values(by=['price_diff'], ascending=True)
    
    return df_shopee.iloc[0]['name']

# df_tt = df_tt[df_tt['min_price'] != 'failed']
# df_tt[df_tt['max_price'] != 'failed']['max_price'] = 0
# replace all max_price = failed with zero
df_tt['max_price'] = df_tt['max_price'].apply(lambda x: 1 if x == 'failed' else x)

df_tt['name'] = df_tt['name'].apply(lambda x: x.replace('_', ' '))
df_shopee['name'] = df_shopee['name'].apply(lambda x: x.replace('_', ' '))
df_tt['name'] = df_tt['name'].apply(lambda x: x.replace('  ', ' '))
df_shopee['name'] = df_shopee['name'].apply(lambda x: x.replace('  ', ' '))

df_tt["shopee_name"] = df_tt.apply(lambda x: find_most_similar_name(x['name'], x['max_price'], df_shopee), axis=1)
df_tt["shopee_id"] = df_tt['shopee_name'].apply(lambda x: df_shopee[df_shopee['name'] == x]['product_id'].values[0] if len(df_shopee[df_shopee['name'] == x]['product_id'].values) > 0 else None)

columns = ["product_id","shopee_id"]
df_tt.to_csv(f"data/{brand}/matched.csv", index=False, columns=columns)


In [ ]:
names = input("Enter names: ").split(';')

for name in names:
    print(name.upper())